In [1]:
import cv2
cv2.__version__

'4.5.1'

In [2]:
# MPII에서 각 파트 번호, 선으로 연결될 POSE_PAIRS
BODY_PARTS = { "Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
                ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
                ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
                ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]
    
# 각 파일 path
protoFile = "E:/data/point/openpose/models/pose/coco/pose_deploy_linevec.prototxt"
weightsFile = "E:/data/point/openpose/models/pose/coco/pose_iter_440000.caffemodel"
 
# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [3]:
# 이미지 읽어오기
image = cv2.imread("E:/data/point/openpose/woman.jpg")

# frame.shape = 불러온 이미지에서 height, width, color 받아옴
imageHeight, imageWidth, _ = image.shape
 
# network에 넣기위해 전처리
inpBlob = cv2.dnn.blobFromImage(image, 1.0 / 255, (imageWidth, imageHeight), (0, 0, 0), swapRB=False, crop=False)
 
# network에 넣어주기
net.setInput(inpBlob)

# 결과 받아오기
output = net.forward()

# output.shape[0] = 이미지 ID, [1] = 출력 맵의 높이, [2] = 너비
H = output.shape[2]
W = output.shape[3]
print("이미지 ID : ", len(output[0]), ", H : ", output.shape[2], ", W : ",output.shape[3]) # 이미지 ID

# 키포인트 검출시 이미지에 그려줌
points = []
for i in range(0,15):
    # 해당 신체부위 신뢰도 얻음.
    probMap = output[0, i, :, :]
 
    # global 최대값 찾기
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

    # 원래 이미지에 맞게 점 위치 변경
    x = (imageWidth * point[0]) / W
    y = (imageHeight * point[1]) / H

    # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
    if prob > 0.1 :    
        cv2.circle(image, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)       # circle(그릴곳, 원의 중심, 반지름, 색)
        cv2.putText(image, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
        points.append((int(x), int(y)))
    else :
        points.append(None)

cv2.imshow("Output-Keypoints",image)
cv2.waitKey(0)

# 이미지 복사
imageCopy = image

# 각 POSE_PAIRS별로 선 그어줌 (머리 - 목, 목 - 왼쪽어깨, ...)
for pair in POSE_PAIRS:
    partA = pair[0]             # Head
    partA = BODY_PARTS[partA]   # 0
    partB = pair[1]             # Neck
    partB = BODY_PARTS[partB]   # 1
    
    #print(partA," 와 ", partB, " 연결\n")
    if points[partA] and points[partB]:
        cv2.line(imageCopy, points[partA], points[partB], (0, 255, 0), 2)


cv2.imshow("Output-Keypoints",imageCopy)
cv2.waitKey(0)
cv2.destroyAllWindows()

이미지 ID :  57 , H :  161 , W :  240


In [6]:
import pandas as pd

aa = pd.read_csv('E:data/aa.csv')
aa.fillna(0, inplace=True)
aa

,file_name,body,x,y
0,737-3-5-37-Z94_E-0000023.jpg,Nose,793,469
1,737-3-5-37-Z94_E-0000023.jpg,Neck,876,516
2,737-3-5-37-Z94_E-0000023.jpg,RShoulder,918,516
3,737-3-5-37-Z94_E-0000023.jpg,RElbow,960,469
4,737-3-5-37-Z94_E-0000023.jpg,RWrist,1001,422
...,...,...,...,...
45479,0,LHeel,1210,798
45480,0,RBigToe,1252,868
45481,0,RSmallToe,1210,868
45482,0,RHeel,1168,845


In [7]:
cc = []
with open('E:/data/aaaa.txt') as f:
    for i in f:
       cc.append(i.split()[0])

In [8]:
count = 0
for i, j in enumerate(aa['body']):
    if aa['file_name'][i] == 0 and j=='Nose':
        aa['file_name'][i] = cc[count]
        count += 1

In [9]:
for i, j  in enumerate(aa['body']):
    if len(str(j)) >15:
        aa.drop(i, axis=0, inplace=True)
        # print(aa[i])
aa.reset_index(inplace=True)
del aa['index']

In [10]:
for i, j  in enumerate(aa['body']):
    if j==0:
        aa.drop(i, axis=0, inplace=True)
aa.reset_index(inplace=True)
del aa['index']

In [11]:
for i, j  in enumerate(aa['file_name']):
    if j==0:
        aa['file_name'][i] = aa['file_name'][i-1]

In [12]:
# aa.set_index('file_name')
aa

,file_name,body,x,y
0,737-3-5-37-Z94_E-0000023.jpg,Nose,793,469
1,737-3-5-37-Z94_E-0000023.jpg,Neck,876,516
2,737-3-5-37-Z94_E-0000023.jpg,RShoulder,918,516
3,737-3-5-37-Z94_E-0000023.jpg,RElbow,960,469
4,737-3-5-37-Z94_E-0000023.jpg,RWrist,1001,422
...,...,...,...,...
40143,649-2-4-32-Z148_D-0000023.jpg,LSmallToe,1335,798
40144,649-2-4-32-Z148_D-0000023.jpg,LHeel,1210,798
40145,649-2-4-32-Z148_D-0000023.jpg,RBigToe,1252,868
40146,649-2-4-32-Z148_D-0000023.jpg,RSmallToe,1210,868


In [41]:
# aa.to_csv('E:/data/test_csv.csv', encoding='utf-8', index=False)
# 이상치 처리해야된다!!!
aa = pd.read_csv('E:/data/test_csv.csv', encoding='utf-8')
aa

,file_name,body,x,y
0,737-3-5-37-Z94_E-0000023.jpg,Nose,793.0,469.0
1,737-3-5-37-Z94_E-0000023.jpg,Neck,876.0,516.0
2,737-3-5-37-Z94_E-0000023.jpg,RShoulder,918.0,516.0
3,737-3-5-37-Z94_E-0000023.jpg,RElbow,960.0,469.0
4,737-3-5-37-Z94_E-0000023.jpg,RWrist,1001.0,422.0
...,...,...,...,...
40000,649-2-4-32-Z148_D-0000023.jpg,LSmallToe,1335.0,798.0
40001,649-2-4-32-Z148_D-0000023.jpg,LHeel,1210.0,798.0
40002,649-2-4-32-Z148_D-0000023.jpg,RBigToe,1252.0,868.0
40003,649-2-4-32-Z148_D-0000023.jpg,RSmallToe,1210.0,868.0


In [42]:
aa.dropna(inplace=True)
aa.isnull().sum()

file_name    0
body         0
x            0
y            0
dtype: int64

In [43]:
aa.reset_index(inplace=True)
del aa['index']
aa

,file_name,body,x,y
0,737-3-5-37-Z94_E-0000023.jpg,Nose,793.0,469.0
1,737-3-5-37-Z94_E-0000023.jpg,Neck,876.0,516.0
2,737-3-5-37-Z94_E-0000023.jpg,RShoulder,918.0,516.0
3,737-3-5-37-Z94_E-0000023.jpg,RElbow,960.0,469.0
4,737-3-5-37-Z94_E-0000023.jpg,RWrist,1001.0,422.0
...,...,...,...,...
39995,649-2-4-32-Z148_D-0000023.jpg,LSmallToe,1335.0,798.0
39996,649-2-4-32-Z148_D-0000023.jpg,LHeel,1210.0,798.0
39997,649-2-4-32-Z148_D-0000023.jpg,RBigToe,1252.0,868.0
39998,649-2-4-32-Z148_D-0000023.jpg,RSmallToe,1210.0,868.0


In [44]:
key_body ={}
for i in range(len(aa)):
    # if aa['file_name'][i]
    if i % 25 == 0:
        key_body[aa['file_name'][i]] = [[aa['body'][i], aa['x'][i], aa['y'][i]]]
    else:
        key_body[aa['file_name'][i]].append([aa['body'][i], aa['x'][i], aa['y'][i]])


In [45]:
len(key_body.keys())

1600

In [46]:
df_sub = pd.read_csv('E:/data/sample_submission.csv')
df = pd.DataFrame(columns=df_sub.columns)
df['image'] = key_body.keys()
# df.iloc[:, 1:] = all_predictions
# df.set_index('image', inplace=True)
df

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,737-3-5-37-Z94_E-0000023.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,737-3-5-37-Z94_C-0000017.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,713-3-5-35-Z94_A-0000005.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,769-3-5-40-Z94_C-0000011.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,729-3-5-36-Z94_C-0000013.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,786-3-5-41-Z94_C-0000007.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,649-2-4-32-Z148_C-0000009.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1597,698-3-5-34-Z94_A-0000003.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1598,665-2-4-33-Z148_C-0000025.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
data_frame = pd.DataFrame(data = key_body.keys())
data_frame

,0
0,737-3-5-37-Z94_E-0000023.jpg
1,737-3-5-37-Z94_C-0000017.jpg
2,713-3-5-35-Z94_A-0000005.jpg
3,769-3-5-40-Z94_C-0000011.jpg
4,729-3-5-36-Z94_C-0000013.jpg
...,...
1595,786-3-5-41-Z94_C-0000007.jpg
1596,649-2-4-32-Z148_C-0000009.jpg
1597,698-3-5-34-Z94_A-0000003.jpg
1598,665-2-4-33-Z148_C-0000025.jpg


In [48]:
# key_body['737-3-5-37-Z94_E-0000023.jpg']

[['Nose', 793.0, 469.0],
 ['Neck', 876.0, 516.0],
 ['RShoulder', 918.0, 516.0],
 ['RElbow', 960.0, 469.0],
 ['RWrist', 1001.0, 422.0],
 ['LShoulder', 834.0, 540.0],
 ['LElbow', 751.0, 469.0],
 ['LWrist', 709.0, 375.0],
 ['MidHip', 876.0, 680.0],
 ['RHip', 876.0, 680.0],
 ['RKnee', 834.0, 751.0],
 ['RAnkle', 960.0, 798.0],
 ['LHip', 834.0, 704.0],
 ['LKnee', 709.0, 751.0],
 ['LAnkle', 751.0, 868.0],
 ['REye', 834.0, 493.0],
 ['LEye', 834.0, 446.0],
 ['REar', 876.0, 469.0],
 ['LEar', 834.0, 469.0],
 ['LBigToe', 709.0, 892.0],
 ['LSmallToe', 709.0, 892.0],
 ['LHeel', 793.0, 892.0],
 ['RBigToe', 960.0, 798.0],
 ['RSmallToe', 960.0, 798.0],
 ['RHeel', 1001.0, 798.0]]

In [49]:
for i, j in enumerate(df['image']):
    df.loc[i][1] = key_body[j][0][1]
    df.loc[i][2] = key_body[j][0][2]
    df.loc[i][3] = key_body[j][16][1]
    df.loc[i][4] = key_body[j][16][2]
    df.loc[i][5] = key_body[j][15][1]
    df.loc[i][6] = key_body[j][15][2]
    df.loc[i][7] = key_body[j][18][1]
    df.loc[i][8] = key_body[j][18][2]
    df.loc[i][9] = key_body[j][17][1]
    df.loc[i][10] = key_body[j][17][2]
    df.loc[i][11] = key_body[j][5][1]
    df.loc[i][12] = key_body[j][5][2]
    df.loc[i][13] = key_body[j][2][1]
    df.loc[i][14] = key_body[j][2][2]
    df.loc[i][15] = key_body[j][6][1]
    df.loc[i][16] = key_body[j][6][2]
    df.loc[i][17] = key_body[j][3][1]
    df.loc[i][18] = key_body[j][3][2]
    df.loc[i][19] = key_body[j][7][1]
    df.loc[i][20] = key_body[j][7][2]
    df.loc[i][21] = key_body[j][4][1]
    df.loc[i][22] = key_body[j][4][2]
    df.loc[i][23] = key_body[j][12][1]
    df.loc[i][24] = key_body[j][12][2]
    df.loc[i][25] = key_body[j][9][1]
    df.loc[i][26] = key_body[j][9][2]
    df.loc[i][27] = key_body[j][13][1]
    df.loc[i][28] = key_body[j][13][2]
    df.loc[i][29] = key_body[j][10][1]
    df.loc[i][30] = key_body[j][10][2]
    df.loc[i][31] = key_body[j][14][1]
    df.loc[i][32] = key_body[j][14][2]
    df.loc[i][33] = key_body[j][11][1]
    df.loc[i][34] = key_body[j][11][2]
    df.loc[i][35] = key_body[j][1][1]
    df.loc[i][36] = key_body[j][1][2]
    df.loc[i][37] = key_body[j][7][1] + 40
    df.loc[i][38] = key_body[j][7][2] - 25
    df.loc[i][39] = key_body[j][4][1] + 40
    df.loc[i][40] = key_body[j][4][2] - 25
    df.loc[i][41] = key_body[j][8][1] + 10
    df.loc[i][42] = key_body[j][8][2] - 35
    df.loc[i][43] = key_body[j][8][1]
    df.loc[i][44] = key_body[j][8][2]
    df.loc[i][45] = key_body[j][19][1]
    df.loc[i][46] = key_body[j][19][2]   
    df.loc[i][47] = key_body[j][22][1]
    df.loc[i][48] = key_body[j][22][2]


In [50]:
df.to_csv('E:/data/openpose_v5.csv', encoding='utf-8', index=False)